In [1]:
import sys
sys.path.insert(0, '../../')

In [12]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
from copy import deepcopy

# 小试一下

In [3]:
# 实例化 cerebro
cerebro = bt.Cerebro()
# 打印初始资金
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# 启动回测
cerebro.run()
# 打印回测完成后的资金
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00


# 一、数据准备

## 1.1 读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

---

### 什么是 `openinterest`（未平仓合约量）？

- **`openinterest`** 是“未平仓合约数量”的意思，主要用于 **期货（futures）和期权（options）** 市场。
- 它表示到目前为止，市场上还有多少份**尚未平仓或交割的合约**。
  - 当有新的买家和卖家开立新仓位时，`openinterest` 会**增加**。
  - 当已有交易者平仓（买入方卖出或卖出方买回）时，`openinterest` 会**减少**。

---

### 为什么 Backtrader 要求这个字段？

- Backtrader 是一个通用的回测框架，不仅支持股票，也支持期货、期权等衍生品。
- 因此，它默认的数据格式中包含了 `openinterest` 这一列，以兼容衍生品数据。

---

### 如果我只做股票交易，怎么办？

- **股票没有“未平仓合约”这个概念**，所以对股票来说，`openinterest` **没有意义**。
- 在这种情况下，你应该将 `openinterest` 列全部设为 **0**。
- 例如，在你的代码中这样处理是完全正确的：
  ```python
  data_.loc[:, ['volume', 'openinterest']] = data_.loc[:, ['volume', 'openinterest']].fillna(0)
  ```

---

### 总结

| 市场类型 | 是否需要 `openinterest` | 建议值 |
|--------|------------------|-------|
| 股票（A股、美股等） | ❌ 不需要 | 填 `0` |
| 期货、期权 | ✅ 需要 | 填真实数据 |

所以，如果你只交易股票，完全可以忽略 `openinterest`，把它设为 0 即可，不会影响回测结果。

In [6]:
daily_price = pd.read_csv("../../datas/daily_price.csv", parse_dates=['datetime'])
daily_price.head()

,datetime,sec_code,open,high,low,close,volume,openinterest
0,2019-01-02,600466.SH,33.064891,33.496709,31.954503,32.386321,10629352,0
1,2019-01-02,603228.SH,50.660230,51.458513,50.394136,51.120778,426147,0
2,2019-01-02,600315.SH,148.258423,150.480132,148.258423,149.558935,2138556,0
3,2019-01-02,000750.SZ,49.512579,53.154883,48.715825,51.561375,227557612,0
4,2019-01-02,002588.SZ,36.608672,36.608672,35.669988,35.763857,2841517,0


In [7]:
daily_price.query("sec_code=='600466.SH'").head()

,datetime,sec_code,open,high,low,close,volume,openinterest
0,2019-01-02,600466.SH,33.064891,33.496709,31.954503,32.386321,10629352,0
546,2019-01-03,600466.SH,32.262944,32.941515,31.399309,31.831127,8602646,0
1211,2019-01-04,600466.SH,31.399309,33.558397,31.337621,33.496709,12768116,0
1700,2019-01-07,600466.SH,33.496709,34.360344,33.373332,33.620085,10584321,0
2136,2019-01-08,600466.SH,33.311644,34.113591,32.694762,33.743462,10012902,0


In [8]:
# 以 datetime 为 index，类型为 datetime 或 date 类型，Datafeeds 默认情况下是将 index 匹配给 datetime 字段；
daily_price = daily_price.set_index(['datetime'])

## 1.2 读取调仓信息表

表内数据说明：

+ trade_date： 调仓期（每月最后一个交易日）;

+ sec_code：持仓成分股；

+ weight：持仓权重。

In [33]:
trade_info = pd.read_csv("../../datas/trade_info.csv", parse_dates=['trade_date'])
trade_info.head()

,trade_date,sec_code,weight
0,2019-01-31,000006.SZ,0.007282
1,2019-01-31,000008.SZ,0.009783
2,2019-01-31,000025.SZ,0.006928
3,2019-01-31,000090.SZ,0.007234
4,2019-01-31,000536.SZ,0.003536


# 二、 选股回测

 选股策略：定期按持仓权重调仓 。

In [35]:
# 回测策略
class TestStrategy(bt.Strategy):
    params = (
        ('buy_stocks', None), # 传入各个调仓日的股票列表和相应的权重
    )
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('{}, {}'.format(dt.isoformat(), txt))

    def __init__(self):
         # 读取调仓日期，即每月的最后一个交易日，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        self.trade_dates = pd.to_datetime(self.p.buy_stocks['trade_date'].unique()).date.tolist()
        self.buy_stock = self.p.buy_stocks # 保留调仓信息
        self.order_list = []  # 记录以往订单，在调仓日要全部取消未成交的订单
        self.buy_stocks_pre = [] # 记录上一期持仓
    
    def next(self):
        # 获取当前的回测时间点
        dt = self.datas[0].datetime.date(0)
        # 打印当前时刻的总资产
        self.log('当前总资产 %.2f' %(self.broker.getvalue()))
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            #取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                print("--------------- 撤销未完成的订单 -----------------")
                for od in self.order_list:
                    # 如果订单未完成，则撤销订单
                    self.cancel(od) 
                 #重置订单列表
                self.order_list = [] 
                
            # 提取当前调仓日的持仓列表
            # 对于简单条件（如单值筛选），直接使用布尔索引（df[df['col'] == value]）通常更快、更清晰
            buy_stocks_data = self.buy_stock[self.buy_stock['trade_date'] == pd.Timestamp(dt)]
            #buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt.isoformat()}'")
            long_list = buy_stocks_data['sec_code'].tolist()
            print('long_list', long_list)  # 打印持仓列表
            
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock)
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data)  
                        self.order_list.append(od) # 记录卖出订单

            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"sec_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)
                
            self.buy_stocks_pre = long_list  # 保存此次调仓的股票列表
        
    #订单日志    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm,
                     order.executed.size,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref,
                         order.executed.price,
                         order.executed.value,
                         order.executed.comm,
                         order.executed.size,
                         order.data._name))

In [36]:
# 实例化大脑
cerebro_ = bt.Cerebro() 

# 按股票代码，依次循环传入数据
for stock in tqdm(daily_price['sec_code'].unique()):
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique())
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].ffill()
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
    cerebro_.adddata(datafeed, name=stock)
print(f"All stock Done !") 


100%|██████████| 510/510 [00:06<00:00, 76.52it/s]

All stock Done !


In [37]:
cerebro = deepcopy(cerebro_)  # 深度复制已经导入数据的 cerebro_，避免重复导入数据 
# 初始资金 100,000,000    
cerebro.broker.setcash(100000.0) 
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003) 
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001) 
# 添加策略
cerebro.addstrategy(TestStrategy, buy_stocks=trade_info) # 通过修改参数 buy_stocks ，使用同一策略回测不同的持仓列表
# 添加分析器
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.003, annualize=True, _name='_SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
# 添加观测器
cerebro.addobserver(bt.observers.Value)  # 查看账户资产变动

# 启动回测
result = cerebro.run()

2019-01-02, 当前总资产 100000.00
2019-01-03, 当前总资产 100000.00
2019-01-04, 当前总资产 100000.00
2019-01-07, 当前总资产 100000.00
2019-01-08, 当前总资产 100000.00
2019-01-09, 当前总资产 100000.00
2019-01-10, 当前总资产 100000.00
2019-01-11, 当前总资产 100000.00
2019-01-14, 当前总资产 100000.00
2019-01-15, 当前总资产 100000.00
2019-01-16, 当前总资产 100000.00
2019-01-17, 当前总资产 100000.00
2019-01-18, 当前总资产 100000.00
2019-01-21, 当前总资产 100000.00
2019-01-22, 当前总资产 100000.00
2019-01-23, 当前总资产 100000.00
2019-01-24, 当前总资产 100000.00
2019-01-25, 当前总资产 100000.00
2019-01-28, 当前总资产 100000.00
2019-01-29, 当前总资产 100000.00
2019-01-30, 当前总资产 100000.00
2019-01-31, 当前总资产 100000.00
--------------2019-01-31 为调仓日----------
long_list ['000006.SZ', '000008.SZ', '000025.SZ', '000090.SZ', '000536.SZ', '000587.SZ', '000598.SZ', '000612.SZ', '000636.SZ', '000656.SZ', '000690.SZ', '000712.SZ', '000766.SZ', '000807.SZ', '000829.SZ', '000877.SZ', '000980.SZ', '000999.SZ', '002002.SZ', '002048.SZ', '002051.SZ', '002074.SZ', '002110.SZ', '002127.SZ', '002128.SZ', '002131.

In [38]:
strat = result[0]
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

--------------- AnnualReturn -----------------
OrderedDict([(2019, 0.23136021328092782), (2020, 0.20744859543672334), (2021, 0.015550720177263555)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', 1.536908114103143)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 136), ('drawdown', 6.4381201511541075), ('moneydown', 10389.99944548888), ('max', AutoOrderedDict([('len', 208), ('drawdown', 20.493180698745974), ('moneydown', 27677.17846390621)]))])
